In [2]:
import pandas as pd
import numpy as np

In [3]:
# load the predictions
predictions = pd.read_csv('/home/oeren/Documents/YZV-NLP/results/with2zemberek.csv',  index_col=[0])
ground_truth = pd.read_csv('/home/oeren/Documents/YZV-NLP/results/test_gold.csv',  index_col=[0])

### CALCULATE ACCURACY BASED ON WORDS

In [7]:
def acc_overall(test_result, testgold):
  correct = 0
  total = 0
  # count number of correctly diacritized words
  for i in range(len(testgold)):
    for m in range(len(testgold[i].split())):
      if test_result[i].split()[m] == testgold[i].split()[m]:
        correct += 1
      total +=1

  return correct / total

In [8]:
accuracy_word = acc_overall(predictions['Sentence'], ground_truth['Sentence'])

In [9]:
accuracy_word

0.9670433145009416

### CALCULATE DIACRTIZATION ERROR RATE (DER)

In [10]:
#calculate the accuracy based on the character level
def DER(test_result, testgold):
  wrong = 0
  total = 0
  # count number of correctly diacritized characters
  for i in range(len(testgold)):
    for m in range(len(testgold[i].split())):
      for j in range(len(testgold[i].split()[m])):
        if testgold[i].split()[m][j] in "ıioöuügğsşcçâêîôûaeIİOÖUÜGĞSŞCÇÂÊÎÔÛAE":
          if test_result[i].split()[m][j] != testgold[i].split()[m][j]:
            wrong += 1
          total +=1

  return wrong / total

In [11]:
der_result = DER(predictions['Sentence'], ground_truth['Sentence'])

In [12]:
der_result

0.014442993095726815

### CALCULATE WORD ERROR RATE

In [18]:
def WER(test_result, testgold):
  wrong = 0
  total = 0
  wrong_words = []
  # count number of correctly diacritized words
  for i in range(len(testgold)):
    for m in range(len(testgold[i].split())):
      if test_result[i].split()[m] != testgold[i].split()[m]:
        wrong_words.append([testgold[i].split()[m], test_result[i].split()[m]])
        wrong += 1
      total +=1

  return wrong / total, wrong_words

In [19]:
wer_result, wrong_words = WER(predictions['Sentence'], ground_truth['Sentence'])

In [20]:
wer_result

0.03295668549905838

##### Create CSV With Wrong Predictions

In [21]:
#create dataframe for wrong words
wrong_words_df = pd.DataFrame(wrong_words, columns=['label', 'prediction'])
wrong_words_df.to_csv('/home/oeren/Documents/YZV-NLP/results/wrong_words2.csv')

### CALCULATE OTHER METRICS

In [22]:
def calculate_metrics(test_result, testgold):
    # Ensure the sequences are of the same length
    assert len(test_result) == len(testgold), "Sequences must be of the same length"
    
    # Initialize counts
    TP = FP = FN = 0
    for i in range(len(testgold)):
        for m in range(len(testgold[i].split())):
            # Iterate over characters in the sequences
            for pred_char, true_char in zip(test_result[i].split()[m], testgold[i].split()[m]):
                if pred_char == true_char and pred_char in 'ıöüğşçâêîôûIÖÜĞŞÇÂÊÎÔÛ':
                    TP += 1  # True positive: Correctly predicted diacritic character
                elif pred_char != true_char and pred_char in 'ıöüğşçâêîôûIÖÜĞŞÇÂÊÎÔÛ':
                    FP += 1  # False positive: Incorrectly predicted diacritic character
                elif pred_char != true_char and true_char in 'ıöüğşçâêîôûIÖÜĞŞÇÂÊÎÔÛ':
                    FN += 1  # False negative: Missed diacritic character
            
    # Calculate precision, recall, and F1 score
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1_score

precision, recall, f1 = calculate_metrics(predictions['Sentence'], ground_truth['Sentence'])
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Precision: 0.98
Recall: 0.96
F1 Score: 0.97
